In [ ]:
sc.list_packages()

In [ ]:
%info

In [ ]:
# {"conf":{"spark.pyspark.python": "python3.7",
#          "spark.pyspark.virtualenv.enabled": "true",
#          "spark.pyspark.virtualenv.type": "native",
#          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"}
# }

In [12]:
# sc.install_pypi_package("scikit-learn")
# sc.install_pypi_package("numpy==1.18.0")
# sc.install_pypi_package("pandas==0.25.1")
# sc.install_pypi_package("requests")
# sc.install_pypi_package("tqdm")
# sc.install_pypi_package("scipy")
# # sc.install_pypi_package("boto3") // No need to run boto3
# sc.install_pypi_package("regex")
# sc.install_pypi_package("Pillow")
# sc.install_pypi_package("h5py==3.0.0")
# sc.install_pypi_package("s3fs")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Get the origin dataframe 

In [13]:
import boto3
from pyspark.sql.types import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
client = boto3.client('s3')

def remove(link):
    new_link = link.split("/")
    new_link = new_link[:-1]
    string = '/'.join(new_link)
    return string

def removeDuplicate(test_list):
    res = []
    for i in test_list:
        if i not in res:
            res.append(i)
    return res
    

def createDataFrame():
    
    sub_list_1 = []

    bucket = client.list_objects_v2(Bucket="kinesis-s3records-16jckkpyv314r", Prefix ='firehose')
    for content in bucket["Contents"]:
        key = content["Key"]
        sub_list_1.append(remove(key))
        
    sub_list_1 = removeDuplicate(sub_list_1)
        
    
    sub_list_2 = []
    
    
    for i in range(len(sub_list_1)):
        sub_list_2.append("s3://kinesis-s3records-16jckkpyv314r/" + sub_list_1[i]+"/")
    

    # Creating the dataframe and append list of object to it
    schema = StructType([
        StructField('caption', StringType(), True),
        StructField('hashtag', StringType(), True),
        StructField('image_url', StringType(), True)
    ])
    df = spark.createDataFrame(spark.sparkContext.emptyRDD(), schema)
    for s3 in sub_list_2:
        df_added = spark.read.json(s3)
        df = df.union(df_added)

    return df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
df = createDataFrame()
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+
|             caption|             hashtag|           image_url|
+--------------------+--------------------+--------------------+
|Read caption 📌

...|["btsatunga", "un...|https://scontent-...|
|The whole event o...|["unitednations",...|https://scontent-...|
|New AirPods Pro a...|["apple", "global...|https://scontent-...|
|Ko Phyu Gyi, a vi...|["Letkoakpin vill...|https://scontent-...|
|Get your hands on...|["peing", "nft", ...|https://scontent-...|
|....................|["btsquotes", "un...|https://scontent-...|
|https://kaderjila...|["HappySunshineDa...|https://scontent-...|
|Where's the lie t...|["bts", "btsedits...|https://scontent-...|
|   #godmorningmonday|["godmorningmonday"]|https://scontent-...|
|Have you register...|["unga", "unwomen...|https://scontent-...|
|New AirPods Pro a...|["apple", "global...|https://scontent-...|
|New AirPods Pro a...|["apple", "global...|https://scontent-...|
|New AirPods Pro a...|["ap

In [16]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

421

# Import needed library

In [17]:
import pandas as pd
import requests
# import regex
from io import StringIO  # python3 (or BytesIO for python2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# import pandas as pd
# import matplotlib.pyplot as plt
# %matplotlib inline
# import matplotlib.image as mpimg
# from PIL import Image
import io
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from pyspark.sql import SparkSession, SQLContext
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.ml.linalg import *
from  pyspark.mllib.linalg.distributed import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import monotonically_increasing_id
from functools import reduce
from pyspark.sql.window import *
import numpy as np
import os
# from tqdm import tqdm
from scipy.spatial.distance import cosine
from pyspark.sql.types import *
import requests
import boto3
from PIL import Image

tf.enable_eager_execution()

np.random.seed(0)
# import s3fs



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Helper function 

In [21]:
# Construct neural net to extract image features 
def construct_neural_network():
    img_shape = (160, 160, 3)

    # Create the base model from the pre-trained model MobileNet V2
    base_model = MobileNetV2(input_shape=img_shape, include_top=False, weights='imagenet')

    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

    neural_network = tf.keras.Sequential([
      base_model,
      global_average_layer,
    ])
    
    return neural_network

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# get image as tensor from url 
def get_image(url):
    r = requests.get(url)
    img = tf.io.decode_image(r.content)
    return img 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
# extract features get the deep features from neural network to use later 
def extract_features(image, neural_network):
    """Return a vector of 1280 deep features for image."""
    images = np.expand_dims(image, axis=0)
    deep_features = neural_network.predict(images)[0]
    return deep_features

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
# transform image to numpy arrays
def prepare_image(img,height = 160,width = 160):
    img = tf.cast(img, tf.float32)
    img = (img/127.5) - 1
    img = tf.image.resize(img, (height, width))
    # Reshape grayscale images to match dimensions of color images
    if img.shape != (160, 160, 3):
        img = tf.concat([img, img, img], axis=2)
    return img.numpy()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
# use for testing on this trial
def get_vector_url(url,neural_network):
    image = prepare_image(get_image(url))
    return extract_features(image,neural_network)

# use for EMR notebook 
def get_vector_img(img,neural_network):
    img = prepare_image(img)
    return extract_features(img,neural_network)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
# helper function for image hashtag generators
def find_neighbor_vectors(features,recommender_df, k=5):
    """Find image features (user vectors) for similar images."""
    rdf = recommender_df.copy()
    rdf['dist'] = rdf['deep_feature'].apply(lambda x: cosine(x, features))
    rdf = rdf.sort_values(by='dist')
    return rdf.head(k)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
# function to read csv file from bucket
def read_csv_file(bucket,key):
    obj = client.get_object(Bucket=bucket, Key=f'dataframe/{key}')
    df = pd.read_csv(io.BytesIO(obj['Body'].read()))
    return df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
# function to write csv file to bucket
def save_df(df,file_name):
    bucket = "kinesis-s3records-16jckkpyv314r"
    csv_buffer = StringIO()
    df.to_csv(csv_buffer)

    s3_resource = boto3.resource('s3')
    s3_resource.Object(bucket, f'dataframe/{file_name}').put(Body=csv_buffer.getvalue())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


# Extract image and text feature from id for the recsys

In [29]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- caption: string (nullable = true)
 |-- hashtag: string (nullable = true)
 |-- image_url: string (nullable = true)

In [30]:
# get the pandas version for creating img and hashtags datasets
df_pd = df.toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
df_pd.caption[0]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'Read caption 📌\n\nThese ANGELS are constantly creating history, "and there\'s no looking back"💪\n.\nGoshhh its day 3 and I can\'t stop screaming about how much proud I am for our boys😭💜\n.\nThey were singing at dancing at UN headquarters , mann its not joke anymore \nI proudly say this "YES THEY ARE THE KINGS OF THIS ERA"\n.\n.\nPLEASE DON\'T REPOST WITHOUT CREDITS 💮\n.\n\n#btsatunga #unga #btsaspresidentenvoy #dramaqueen #BTS #taehyung\n#v #namjoon #kimnamjoon #kimnamjoonkimseokjinminyoongijunghoseokparkjiminkimtaehyungjeonjungkookbts #jimin #jungkook #yoongi #hobi #hoseak #jin #seokjin #kpop #army #btsarmy #btsarmyforever #butter #btsbutter #hybe #hybelabels #buttermv #bangtanboys #btsedit #btslove #purpleyou💜'

In [32]:
import re
def find_hashtag(caption):
    to_replace = ["\n", ".", ","]
    temp = caption
    for i in to_replace:
        temp = temp.replace(i,"")
    hashtags = re.findall("#[A-Za-z]+", temp)
    
    return hashtags    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
df_pd["hashtags"] = df_pd.caption.apply(lambda x: find_hashtag(x))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
df_pd["hashtags"]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0      [#btsatunga, #unga, #btsaspresidentenvoy, #dra...
1      [#unitednations, #unga, #bts, #bangtanboys, #b...
2      [#apple, #globalarmsembargo, #unga, #godmornin...
3      [#Letkoakpin, #Natmauk, #globalarmsembargo, #s...
4      [#peing, #nft, #emmys, #ps, #amici, #nowplayin...
                             ...                        
416    [#happyseungminday, #happybirthdayseungmin, #s...
417    [#NFTcollectibles, #nftartgallery, #nftcollect...
418              [#seungminstraykids, #seungminbirthday]
419    [#Original, #song, #straykids, #straykidsseung...
420                  [#jinyoung, #allrounderjinyoungday]
Name: hashtags, Length: 421, dtype: object

In [35]:
# Create a flattened list of all hashtags
all_hashtags = [hashtag for hashtags in df_pd['hashtags'] for hashtag in hashtags]

# Coerce to a set to remove duplicate entries
# Sort to ensure reproducibility of results
all_hashtags = sorted(list(set(all_hashtags)))

# Build lookup for finding hashtag number based on hashtag name
hashtag_lookup = {hashtag: i for i, hashtag in enumerate(all_hashtags)}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
# create dataframe to save all hashtags
hashtags_df = pd.DataFrame.from_dict(hashtag_lookup, orient='index')
hashtags_df.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                    0
#ARMYvaccinatedtoo  0
#ATEEZBB            1
#AdBor              2
#Adbor              3
#AltPunks           4

In [37]:
hashtags_df = hashtags_df.reset_index()
hashtags_df.columns = ['hashtag', 'id']
hashtags_df.index = hashtags_df['id']
hashtags_df.drop('id', axis=1, inplace=True)
hashtags_df.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

               hashtag
id                    
0   #ARMYvaccinatedtoo
1             #ATEEZBB
2               #AdBor
3               #Adbor
4            #AltPunks

In [38]:
# Save hashtags dataframe 
save_df(hashtags_df,"hashtags.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
df_pd["img_name"] = df_pd.index.to_series().apply(lambda x: f"img_{str(x)}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
neural_net = construct_neural_network()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
def get_deep_feature(url,neural_net):
    try:
        return get_vector_url(url,neural_net)
    except:
        pass

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
# funtion to loading the deep features from saved dataframe
def create_deepfeatures(index,original_dataframe,saved_dataframe,neural_net):
    if index <= saved_dataframe.index[-1]:
        return saved_dataframe.loc[index,"deep_feature"]
    else:
        try:
            url = original_dataframe.loc[index,"image_url"]
            return get_deep_feature(url,neural_net)
        except:
            return np.zeros(1280)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
# load saved deep_features 
import ast
bucketname = "kinesis-s3records-16jckkpyv314r"
keyname = "df_features.csv"
saved_df_features = read_csv_file(bucketname,keyname)

saved_df_features["deep_feature"] = saved_df_features["deep_feature"].apply(lambda x: np.fromstring(x[1:-1],sep=", ") )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
saved_df_features.iloc[294]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Unnamed: 0                                                    294
caption         Take the opportunity to multiply your money by...
hashtag         ["nft", "nfts", "nftart", "nftartist", "nftcom...
image_url       https://scontent-ort2-1.cdninstagram.com/v/t51...
hashtags        ['#nft', '#nfts', '#nftart', '#nftartist', '#n...
img_name                                                  img_294
deep_feature    [0.4041658341884613, 0.40114936232566833, 0.03...
Name: 294, dtype: object

In [45]:
type(saved_df_features.deep_feature[0])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'numpy.ndarray'>

In [46]:
saved_df_features

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

     Unnamed: 0  ...                                       deep_feature
0             0  ...  [0.0, 0.0, 0.865784764289856, 0.00069944956339...
1             1  ...  [0.879599928855896, 0.23585304617881775, 1.039...
2             2  ...  [0.0, 0.0, 0.7003890872001648, 0.1560206115245...
3             3  ...  [0.0075896806083619595, 0.0, 0.532040476799011...
4             4  ...  [0.0, 0.11799856275320053, 0.0, 0.088474199175...
..          ...  ...                                                ...
416         416  ...  [0.06956370174884796, 0.039646413177251816, 0....
417         417  ...  [3.126276969909668, 0.0, 0.0, 0.0, 0.209734916...
418         418  ...  [0.15421395003795624, 0.655612051486969, 0.333...
419         419  ...  [0.2562243640422821, 0.0, 0.0, 2.1857922077178...
420         420  ...  [0.0, 0.0, 0.0, 3.2210769653320312, 1.97364747...

[421 rows x 7 columns]

In [47]:
df_pd["deep_feature"] = df_pd.index.to_series().apply(lambda x:  create_deepfeatures(x, df_pd,saved_df_features ,neural_net))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
df_save = df_pd.copy()
df_save["deep_feature"] = df_save.deep_feature.apply(lambda x: str(x.tolist()))
save_df(df_save,"df_features.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
# construct dataframe for recsys of hashtag_id and image_id 
hashtag_rec_data = []
for i in df_pd.index:
    hashtag_list = df_pd.loc[i, 'hashtags']
    for hashtag in hashtag_list:
        hashtag_rec_data.append(
            {'image_id': i,
             'hashtag_id': hashtag_lookup[hashtag],
             'rating': 1}
        )
hashtag_rec_data_df = pd.DataFrame(hashtag_rec_data)
hashtag_rec_data_df.tail()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      image_id  hashtag_id  rating
4775       419        1382       1
4776       419         501       1
4777       419         812       1
4778       420         841       1
4779       420         163       1

In [50]:
save_df(hashtag_rec_data_df,"rec_hashtags.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
als = ALS(userCol='image_id',
          itemCol='hashtag_id',
          implicitPrefs=True,
          alpha=40)

als.setSeed(0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ALS_1323658ac7b1

In [52]:
sqlcontext = SQLContext(sc)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
rec_hashtags = sqlcontext.read.csv("s3://kinesis-s3records-16jckkpyv314r/dataframe/rec_hashtags.csv",header = True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
rec_hashtags.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------+----------+------+
|_c0|image_id|hashtag_id|rating|
+---+--------+----------+------+
|  0|       0|       316|     1|
|  1|       0|      1477|     1|
|  2|       0|       314|     1|
|  3|       0|       556|     1|
|  4|       0|        13|     1|
|  5|       0|      1423|     1|
|  6|       0|      1492|     1|
|  7|       0|      1045|     1|
|  8|       0|       884|     1|
|  9|       0|       885|     1|
| 10|       0|       839|     1|
| 11|       0|       852|     1|
| 12|       0|      1549|     1|
| 13|       0|       780|     1|
| 14|       0|       786|     1|
| 15|       0|       840|     1|
| 16|       0|      1309|     1|
| 17|       0|       900|     1|
| 18|       0|       183|     1|
| 19|       0|       310|     1|
+---+--------+----------+------+
only showing top 20 rows

In [55]:
rec_hashtags.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4780

In [56]:
rec_hashtags.toPandas().tail(4)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

       _c0 image_id hashtag_id rating
4776  4776      419        501      1
4777  4777      419        812      1
4778  4778      420        841      1
4779  4779      420        163      1

In [57]:
rec_hashtags.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _c0: string (nullable = true)
 |-- image_id: string (nullable = true)
 |-- hashtag_id: string (nullable = true)
 |-- rating: string (nullable = true)

In [58]:
for i in ["image_id","hashtag_id","rating"]:
    rec_hashtags = rec_hashtags.withColumn(i, rec_hashtags[i].cast(IntegerType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
als_model = als.fit(rec_hashtags)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
# get image and hashtags features extracted from als
img_features = als_model.userFactors.toPandas()
hashtag_features = als_model.itemFactors.toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [61]:
img_features = img_features.set_index("id")
hashtag_features = hashtag_features.set_index("id")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
img_features

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                              features
id                                                    
0    [0.13956701755523682, 0.7346398234367371, 0.05...
10   [-0.06916030496358871, -0.3364502191543579, 0....
20   [0.11296151578426361, 0.4349663555622101, 0.03...
30   [-0.2047702521085739, 0.2715473473072052, 0.06...
40   [0.21806775033473969, -0.2691774070262909, -0....
..                                                 ...
379  [-0.26747846603393555, 0.2883451581001282, -0....
389  [-0.12186341732740402, 0.1351088285446167, 0.0...
399  [-0.4461362659931183, -0.07086769491434097, 0....
409  [0.15092189610004425, -0.3283292055130005, -0....
419  [0.0952983871102333, 0.5853119492530823, 0.498...

[415 rows x 1 columns]

In [63]:
hashtag_features

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                               features
id                                                     
0     [-0.15813808143138885, -0.1324387788772583, -0...
10    [-0.0003790539049077779, 0.2680297791957855, -...
20    [0.3046354055404663, -0.26104578375816345, 0.1...
30    [-0.09749610722064972, 0.14251653850078583, -0...
40    [0.27278828620910645, -0.0023461305536329746, ...
...                                                 ...
1519  [-0.17712463438510895, -0.20465600490570068, 0...
1529  [-0.038235072046518326, 0.06197502836585045, -...
1539  [0.12802544236183167, -0.09011849015951157, 0....
1549  [0.21435751020908356, 0.43510493636131287, 0.0...
1559  [-0.11038688570261002, -0.16956259310245514, -...

[1560 rows x 1 columns]

In [64]:
# save features for later used 
save_df(img_features,"img_features.csv")
save_df(hashtag_features,"hashtag_features.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [65]:
def fetch_image_from_s3_to_array(client,bucket, key):
    """Fetches an image from S3 and returns a numpy array."""
    response = client.get_object(Bucket=bucket, Key=key)
    body = response["Body"]
    data = body.read()
    f = BytesIO(data)
    image = Image.open(f)
    image_data = np.asarray(image)
    return image_data
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
def get_hashtags(df_features, hashtags_df,images_features,hashtag_features,neural_net = neural_net, url = None,client = client ,bucket=None,key = None,number = 20):
    if url:
        deep_features = get_vector_url(url,neural_net)
        print(type(deep_features[0]))
    else:
        image = fetch_image_from_s3_to_array(client,bucket,key)
        deep_features = get_vector_img(image,neural_net)
    df_rec = df_features.copy()
    chosen = find_neighbor_vectors(deep_features,df_rec)
    list_features_combine = [images_features.loc[i,"features"] for i in chosen.index.tolist()]
    average_features = np.asarray(list_features_combine).mean(axis = 0)
    hashtags = hashtag_features.copy()
    hashtags["dot_product"] = hashtags.features.apply(lambda x: np.asarray(x).dot(average_features))
    final_recs = hashtags.sort_values(by='dot_product', ascending=False).head(number)
    # Look up hashtags by their numeric IDs
    output = []
    for hashtag_id in final_recs.index.values:
        output.append(hashtags_df.iloc[hashtag_id]['hashtag'])

    return output

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
url = "https://image.thanhnien.vn/1024/uploaded/binhhh/2020_09_28/bts-2_kbet.jpg"
a = get_hashtags(df_pd,hashtags_df,img_features,hashtag_features,url =url )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'numpy.float32'>

In [68]:
a

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['#chen', '#kpop', '#exo', '#happybirthday', '#exol', '#love', '#kimjongdae', '#chenexo', '#happychenday', '#weareoneexo', '#jongdae', '#weareone', '#baekhyun', '#kimjongdaeexo', '#sehun', '#chanyeol', '#lay', '#kai', '#xiumin', '#suho']

In [73]:
bucketname = "kinesis-s3records-16jckkpyv314r"
keyname = "df_features.csv"
df_features = read_csv_file(bucketname,keyname)
df_features["deep_feature"] = df_features["deep_feature"].apply(lambda x: np.fromstring(x[1:-1],sep=", ") )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
hashtags_file = "hashtags.csv"
hashtags_df = read_csv_file(bucketname,hashtags_file)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
images_features_file = "img_features.csv"
images_features = read_csv_file(bucketname,images_features_file)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
hashtag_features_file = "hashtag_features.csv"
hashtag_features = read_csv_file(bucketname,hashtag_features_file)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
# images_features.features

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0      [0.13956701755523682, 0.7346398234367371, 0.05...
1      [-0.06916030496358871, -0.3364502191543579, 0....
2      [0.11296151578426361, 0.4349663555622101, 0.03...
3      [-0.2047702521085739, 0.2715473473072052, 0.06...
4      [0.21806775033473969, -0.2691774070262909, -0....
                             ...                        
410    [-0.26747846603393555, 0.2883451581001282, -0....
411    [-0.12186341732740402, 0.1351088285446167, 0.0...
412    [-0.4461362659931183, -0.07086769491434097, 0....
413    [0.15092189610004425, -0.3283292055130005, -0....
414    [0.0952983871102333, 0.5853119492530823, 0.498...
Name: features, Length: 415, dtype: object